In [1]:
import pandas as pd
marks_excel = pd.read_excel('original.xlsx')
sub_excel = pd.read_excel('sublist.xlsx')

import numpy as np


In [2]:
# create marks and sub data
marks = marks_excel.copy()
sub_data = sub_excel.copy()
sub_data = sub_data.T
sub_data.columns = ['Subject Code','SUBJECTS','CREDIT HOUR(CH)']


In [3]:
#access by last digit of registeration no in marks 
def student(last_digit):
    regno = 782330220000 + last_digit
    return marks.loc[marks['rn'] == regno]


In [4]:
# select only sub th and sub in marks
def marksheet(last_digit):
    temp = student(last_digit)
    temp = temp.iloc[:,4:] # remove details other than marks
    temp = temp.drop(['en','ne','so','ec','bu','ac','ch','ed','on','oh'],axis=1)
    temp = temp.dropna(axis = 1) # drop unselected and subjects that has no marks
    return temp
# select student regno, roll no, name , dob
def student_info(last_digit):
    temp = student(last_digit)
    temp = temp[['rn','sn','name','dob']]
    temp.columns = ['REGISTERATON NO','SYMBOL NO','NAME','DOB']
    return temp

In [5]:
# helper functions 
def re_scale(num): # scale number in 100 scale
    #caution assumes no theory number assigned to students is less than or equall to 25
    if (num > 25):
        scaled_mark = (num/75)*100
    else:
        scaled_mark = (num/25)*100
    return scaled_mark

def conv_into_gp(num): # convert scaled number into gp
    if (num >= 90):
        gp = 4.0
    elif (num >= 80 and num < 90):
        gp = 3.6
    elif (num >= 70 and num < 80):
        gp = 3.2
    elif (num >= 60 and num < 70):
        gp = 2.8
    elif (num >= 50 and num < 60):
        gp = 2.4
    elif (num >= 40 and num < 50):
        gp = 2.0
    elif (num >= 35 and num < 40):
        gp = 1.6
    elif (num < 35):
        gp = 0
    return gp

def gp_to_letter(num):
    if (num == 4.0):
        l = 'A+'
    elif(num == 3.6):
        l = 'A'
    elif(num == 3.2):
        l = 'B+'
    elif (num == 2.8):
        l = 'B'
    elif (num == 2.4):
        l = 'C+'
    elif (num == 2.0):
        l = 'C'
    elif(num == 1.6):
        l = 'D'
    elif(num == 0):
        l = 'NG'
    return l    

def letter_to_gp(l):
    if (l == 'A+'):
        gp = 4.0
    elif(l == 'A'):
        gp = 3.6
    elif(l == 'B+'):
        gp = 3.2
    elif (l == 'B'):
        gp = 2.8
    elif (l == 'C+'):
        gp = 2.4
    elif (l == 'C'):
        gp = 2.0
    elif(l == 'D'):
        gp = 1.6
    elif(l == 'NG'):
        gp = 0
    return gp    
    

In [6]:
# create grade sheet of student
def gradesheet(last_digit):
    sheet = marksheet(last_digit)
    # rescale
    rsheet = sheet.applymap(re_scale)
    gpsheet = rsheet.applymap(conv_into_gp)
    gpsheet = gpsheet.T
    gpsheet.columns=['GRADE POINT(GP)']
    conditions = [
        (gpsheet['GRADE POINT(GP)'] == 4.0),
        (gpsheet['GRADE POINT(GP)'] == 3.6),
        (gpsheet['GRADE POINT(GP)'] == 3.2),
        (gpsheet['GRADE POINT(GP)'] == 2.8),
        (gpsheet['GRADE POINT(GP)'] == 2.4),
        (gpsheet['GRADE POINT(GP)'] == 2.0),
        (gpsheet['GRADE POINT(GP)'] == 1.6),
        (gpsheet['GRADE POINT(GP)'] == 0),
    ]
    choices = ['A+','A','B+','B','C+','C','D', 'NG']
    gpsheet['GRADE'] = np.select(conditions, choices)
    return gpsheet

In [7]:
# create full grade_sheet except final grade column
def full_grade_sheet(last_digit):
    temp = gradesheet(last_digit)
    temp = pd.concat([sub_data, temp], axis=1)
    temp = temp.dropna(axis = 0)
    return temp

In [8]:
# create final grade column
def final_grade_col(last_digit):
    # select total sub marks
    temp = student(last_digit)
    temp = temp.iloc[:,4:] # remove details other than marks
    temp = temp[["en","ne","so","ec","bu","ac","ch","ed","on","oh"]]
    temp.columns = ["en_th","ne_th","so_th","ec_th","bu_th","ac_th","ch_th","ed_th","on_th","oh_th"]
    temp = temp.dropna(axis = 1) # drop unselected and subjects that has no marks
    temp = temp.applymap(conv_into_gp)
    temp = temp.applymap(gp_to_letter)
    temp = temp.T
    temp.columns = ['FINAL GRADE (FG)']
    return temp

In [9]:
def complete_sheet(last_digit):
    # acess fullgrade sheet without ginal grade col
    sheet1 = full_grade_sheet(last_digit)
    # acess final grade col
    sheet2 = final_grade_col(last_digit)
    #concatenate
    temp = pd.concat([sheet1, sheet2], axis=1)
    temp['REMARKS'] = np.nan
    return temp

In [10]:
# function to calculate GPA
def cal_gpa(last_digit):
    # acess complete sheet
    sheet = complete_sheet(last_digit)
    # acess grade point column and credit hour coloumn
    gpa  = np.sum(sheet['CREDIT HOUR(CH)']*sheet['GRADE POINT(GP)'])/np.sum(sheet['CREDIT HOUR(CH)'])
    return gpa

In [11]:
def prepare(last_digit):
    sheet = complete_sheet(last_digit)
    filename = '0000'+str(last_digit)+".xlsx"
    sheet.to_excel(filename)
    return cal_gpa(last_digit),student_info(last_digit)
    

In [12]:
prepare(5)

(2.977777777777778,
    REGISTERATON NO  SYMBOL NO               NAME        DOB
 4     782330220005   13302279  Amrita Chaurasiya 2062-01-05)